In [1]:
#! pip install -q lalsuite
#! pip install PyCBC
#! pip install joblib

import os
import numpy as np
#from Starter.Detection import Detector, Network
from Stochastic import princess as SP
from Starter.astromodel import AstroModel
from Starter.detection import Detector, Network 
from Individual.individual_analysis import IndividualAnalysis as IA


/home/carole/.local/lib/python3.10/site-packages/pycbc/types/array.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(True)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal as _lal



<div>
<img src="./Wiki/Princess_logo.png" width="300"/>
</div>

# Princess: Guide for a first calculation


This file goes step-by-step through the calculation of an astrophysical background starting from a CBC catalogue.
For mode details about the code structure and basic calculations please visit the README.md file.
Princess as two companioin paper which give more details on the physica behind these calculations.

This toolkit aims to be user friendly and useful to the collaboration. If you have any comments, issues or requests please contact the administrator (_caroleperigois@outlook.com_).

The calculation of the background goes through four main step, defining the four sections of this file.
1. Prepare your model
2. Prepare your detectors and Networks
3. Calculate the background
4. Analyse our background

## 1. Prepare your model: 
All population synthesis codes may have different outputs. In the next steps the code will re-build catalogues insuring that it contains all the required parameters for the next steps of calculation.
Your astrophysical model will be set in a class Princess.Astromodel and takes in entry several parameter.
* `name`: (_str_) is the name you want to use for your model
* `original_path`: (_str_) path to your original astrophysical catalogue
* `sep`: (_str_) separator used in your original catalogue (default is tab)
* `index_column`: (_bool_) does your original file contain a columns with indexes (default is None)
* `flags`: (_dict_) this option allow to differenciate different types of CBCs is a Model. If you add this option you need to set up a dictionnary of the different categories. For example in the checks the original catalogue contain a column called 'flag' wher can be found identifiers 1 for isolated BBH and 2 for cluster BBH. Therefore the dictionnary looks like Flags = {'1': 'Iso', '2':'Cluster'}. In the next steps the code will build two catalogues out from the initial model (default is None).
* `spin_option`: (_str_) name of the option to use for spins.



In [2]:
path_cat_original = './Test.dat'
astromodel = AstroModel(name= 'Princess_Test', original_path = path_cat_original, 
                         sep = "\t", index_column = None, spin_option = 'Rand_dynamics')

0


If your original catalogue do not have header you can set one using the method makeHeader on your model. In the next liste are the labels allowed by the code, please note that for the masse you will need to have or the chirp mass and the mass ratio, or the two masses m1 and m2.
* Mc : Chirp mass in the source frame [Msun]
* q : mass ratio in the source frame
* m1 : mass of the first component (m1>m2) in the source frame [Msun]
* m2 : mass of the secondary component (m2<m1) in the source frame [Msun]
* Xeff : effective spin
* s1 : individual spin factor of the first component
* s2 : individual spin factor of the second component
* theta1 : angle between the first spin component and the angular momentum of the binary [rad]
* theta2 : angle between the second spin component and the angular momentum of the binary [rad] 
* a0 : semi-major axis at the formaiton of the second compact object [Rsun]
* e0 : eccentricity at the formation of the second compact object  
* inc : inclinaison angle [rad]       
* zm : redshift of merger
* zf : redshift of formation  
* Dl : luminosity distance [Mpc]
* flag : this columns cam contain a flag to differenciate sources

Set spin option:
    True if you want to include spin in your calculations of the waveforms and background and you have the spin in your catalogue
    'Zero' if you don't want to use the spin, the code set all spins to 0.
    Model if you want Princess to generate spin values - Option available later -

Finally the MakeCat method generate the catalogue with all requiered parameters

In [3]:
#astromodel.make_catalog()

print('Astromodel loaded and ready :)')

Astromodel loaded and ready :)


## 2. Detectors and Networks
In this part is detailed the context of the study starting by defining the range of frequency `Freq` and the waveforms `WF_approx` to use. In this version the range has to be linear in the future specific function will be added to allow log scale, in particular for LISA band. The available waveforms are the ones define in PyCBC and the analytic one from Ajith2011. The calculation with Ajith waveforms is computationnaly more expensive and therefore not recommended.   

In [4]:
Freq_2G = np.linspace(1, 1000, 991)
Freq_3G = np.linspace(1, 1000, 1000)
WF_approx = "IMRPhenomD"

In Princess two classes has been build for this purpose in the file _Starter/Detection_ in order to define Detectors and combine them to build a Network. The Detector class takes in entry several parameters:
* `name`: (_str_) is the name you give to the detector
* `Pycbc`: (_bool_) True if the sensitivity is available in PyCBC, else False
* `psd_file`: (_str_) name of the sensitivity in PyCBC, or file where your sensitivity is stored
* `freq`: (_np.array_) frequency range of the study

In [5]:
H = Detector(name = 'H', origin = 'Pycbc', configuration = 'H', psd_file = 'aLIGODesignSensitivityP1200087', freq = Freq_2G)
L = Detector(name = 'L', origin = 'Pycbc', configuration = 'L', psd_file = 'aLIGODesignSensitivityP1200087', freq = Freq_2G)
V = Detector(name = 'V', origin = 'Pycbc', configuration = 'V', psd_file = 'AdVDesignSensitivityP1200087', freq = Freq_2G)

In [6]:
ET = Detector(name = 'ET', configuration = 'ET', origin = 'Pycbc', psd_file = 'EinsteinTelescopeP1600143', freq = Freq_3G)
CE1 = Detector(name = 'CE1', origin = 'Princess', configuration = 'H', psd_file = 'CE_40km', freq = Freq_3G)
CE2 = Detector(name = 'CE2', origin = 'Princess', configuration = 'L', psd_file = 'CE_40km', freq = Freq_3G)

The second class Network allow to combine different detectors to build a Network.A Network takes in entry: 
* `net_name`: (_str_) Name of the network
* `compo`: (_list of detectors_) List of the detectors in the network.
* `pic_file`: (_str_) link to the file of the power integrated curve.
* `efficiency`: (_float_) between 0 and 1 define the effective time of observation of the Network. For example during O3a, in the Hanford-Livinstone-Virgo network only 50% of the data can be used with the three pipelines. The rest of the time at least on detector pipeline was unusuable.
* `SNR_thrs`: (_int_ or _float_) Define the SNR threshold for which we assume a source is detectable
* `SNR_sub`: (_int_ or _float_) Define the SNR threshold to substract the sources. For example its commonly assumed that in HLV all source with an SNR above 8 are resolved. However for a reason of parameter uncertainty the calculation of the residual background is done by subtracting only source with a SNR above 12. 


**If only one detector is used in the study it still has to be set as a detector.**

The variable `Networks` gather all the networks used in the study.


In [7]:
HLV_Des = Network(name = 'HLV',compo=[H,L,V], pic_file = 'AuxiliaryFiles/PICs/Design_HLV_flow_10.txt', freq = Freq_2G, efficiency = 0.5,SNR_thrs = 12 )
Networks_2G = [HLV_Des]

In [8]:
ETn = Network(name='ET', compo=[ET], pic_file='AuxiliaryFiles/PICs/ET2CE.txt',SNR_thrs=12)
twoCE = Network(name='2CE', compo=[CE1,CE2], pic_file='AuxiliaryFiles/PICs/ET2CE.txt',SNR_thrs=12)
ET2CE = Network(name='ET2CE', compo=[ET,CE1,CE2], pic_file='AuxiliaryFiles/PICs/ET2CE.txt',SNR_thrs=12)
Networks_3G = [ETn, twoCE, ET2CE]

Finally the method `compute_SNR`, compute the SNR of each sources of the model, and update the catalogue(s) with a new parameter column named by `net_name` containing the SNR in the corresponding network.

In [9]:
#astromodel.compute_SNR_opt(Networks = Networks_2G, freq = Freq_2G, approx = WF_approx)

## 5. Individual analysis 
This package of the code aims to predict individual observation from LIGO-Virgo from a predefined astrophysical model. As for the background computation the user has to predefine his analysis with the _class_ 'IndividualAnalysis'. This class takes in entry the limits of the study and the assumption on GW events observations.

* `name`: (_str_) Name used to label the output.
* `params` : (_dict_) parameters the user want to compare and the range [minimum, maximum, bins]. Default are : {'m1': [0, 100, 100], 'q': [0, 1, 20], 'zm': [0, 5, 30]}
* `iteration` : (_int_) number of iteration to extract the errors on observations
* `Network` : (Network from _class_ 'Network') network to use in the study
* `binary_type` : (_list_ of _str_) type of binaries to be compared. Default is ['BBH', 'BNS', 'NSBH'].
* `pastro_thrs` : (_float_) threshold to select sources from the data for the comparison. Default is 0.
* `SNR_thrs` : (_float_) threshold to select sources from the data for the comparison. Default is 0.
* `FAR_thrs` : (_float_) threshold to select sources from the data for the comparison. Default is 2.
* `iteration` : (_int_) The number of iteration used to set up the position in the sky, this is used to extract an error on the number of detections.

The method `Full_Analysis` is used to start the computation of the analysis.

In [10]:
#analysis = IA(name = 'PRINCESS_check', params =  {'m1': [0, 100, 100], 'm2': [0, 100, 100], 'Mc': [0, 70, 100]},
#                             Networks = Networks_2G, binary_type = ['BBH'], iteration = 1)
#analysis.Full_Analysis(Model = astromodel, update_file = False)

## 4. Calculate the corresponding background:

Prepare the calculation of the background with the class `Princess` :
* `freq`: (_np.array_) frequency range, preferentially the one used since the beginning. In linear scale before the LISA update
* `approx`: (_str_) waveform approximation
* `freq_ref`: (_list of float_) list of frequency of interest for the study, is usually 10Hz for 3G detectors and 25Hz for 2G.

Then the calculation is done by using the method `Omega_pycbc`

In [11]:
Zelda = SP.Princess(Freq_2G, astromodel = astromodel, approx = WF_approx, Omega_ana_freq = [10.,25.], Networks = Networks_2G, inclination = "Rand")
Zelda.Make_Ana_Output()
#Zelda.Omega_pycbc(Networks= Networks_2G)

{'Princess_Test.dat':              Total  HLV
N_source       NaN  NaN
Omg_10.0_Hz    NaN  NaN
Omg_25.0_Hz    NaN  NaN
SNR_Total      NaN  NaN
SNR_Residual   NaN  NaN}


## 4. Analyse the background
This part of the code aims to extract reference values for the predicted background. Usual values are the amplitude at 10 and 25 Hz, the SNR, the number of resolved soures, the ratio of detected sources, and the ratio between residuals and total backgrouns at a reference value.

In [12]:
#E1 = Detector(name = 'E1', configuration = 'E1', origin = 'Pycbc', psd_file = 'EinsteinTelescopeP1600143', freq = Freq_3G)
#E2 = Detector(name = 'E2', configuration = 'E2', origin = 'Pycbc', psd_file = 'EinsteinTelescopeP1600143', freq = Freq_3G)
#E3 = Detector(name = 'E3', configuration = 'E3', origin = 'Pycbc', psd_file = 'EinsteinTelescopeP1600143', freq = Freq_3G)##

#CE1 = Detector(name = 'CE1', origin = 'Princess', configuration = 'CE1', psd_file = 'CE_20km', freq = Freq_3G)
#CE2 = Detector(name = 'CE2', origin = 'Princess', configuration = 'CE2', psd_file = 'CE_40km', freq = Freq_3G)
#ETn = Network(name='ET', compo=[E1,E2,E3], pic_file='AuxiliaryFiles/PICs/ET.txt',SNR_thrs=12, duration = 1.)
#twoCE = Network(name='2CE', compo=[CE1,CE2], pic_file='AuxiliaryFiles/PICs/ET2CE.txt',SNR_thrs=12, duration = 1.)
#ET2CE = Network(name='ET2CE', compo=[E1,E2,E3, CE1,CE2], pic_file='AuxiliaryFiles/PICs/ET2CE.txt',SNR_thrs=12, duration = 1)
#Networks_3G_bkg = [ETn, twoCE, ET2CE]
#Zelda.Analysis(Networks = Networks_3G_bkg)
#Zelda.Write_results()

In [13]:
Zelda.Analysis(Networks = Networks_2G)
Zelda.Write_results()

[ 1.00000000e+00  1.00000000e+00 -9.95682444e-60 -5.97409467e-59
  3.98272978e-58 -1.59309191e-57  8.92131470e-57  6.26021397e-54
  9.86239894e-41  2.26946009e-44  7.27856072e-45  3.93483917e-45
  2.48317636e-45  1.69734768e-45  1.22089920e-45  9.10315160e-46
  6.97503810e-46  5.46286100e-46  4.35787643e-46  3.53210978e-46
  2.90334048e-46  2.41689104e-46  2.03529835e-46  1.73227610e-46
  1.48901511e-46  1.29180271e-46  1.13048826e-46  9.97461513e-47
  8.86934725e-47  7.94465761e-47  7.16604013e-47  6.50647109e-47
  5.94458749e-47  5.46339235e-47  5.04924808e-47  4.69111537e-47
  4.38002442e-47  4.10863754e-47  3.87094195e-47  3.66192578e-47
  3.47746041e-47  3.31407283e-47  3.16885329e-47  3.03935692e-47
  2.92352191e-47  2.81958401e-47  2.72603066e-47  2.64160364e-47
  2.56518427e-47  2.49582935e-47  2.43271841e-47  2.37515080e-47
  2.32250032e-47  2.27423984e-47  2.22989274e-47  2.18905080e-47
  2.15135289e-47  2.11648133e-47  2.08416359e-47  2.05414042e-47
  2.02620290e-47  2.00015

[ 1.00000000e+00  1.00000000e+00 -9.95682444e-60 -5.97409467e-59
  3.98272978e-58 -1.59309191e-57  8.92131470e-57  6.26021397e-54
  9.86239894e-41  2.26946009e-44  7.27856072e-45  3.93483917e-45
  2.48317636e-45  1.69734768e-45  1.22089920e-45  9.10315160e-46
  6.97503810e-46  5.46286100e-46  4.35787643e-46  3.53210978e-46
  2.90334048e-46  2.41689104e-46  2.03529835e-46  1.73227610e-46
  1.48901511e-46  1.29180271e-46  1.13048826e-46  9.97461513e-47
  8.86934725e-47  7.94465761e-47  7.16604013e-47  6.50647109e-47
  5.94458749e-47  5.46339235e-47  5.04924808e-47  4.69111537e-47
  4.38002442e-47  4.10863754e-47  3.87094195e-47  3.66192578e-47
  3.47746041e-47  3.31407283e-47  3.16885329e-47  3.03935692e-47
  2.92352191e-47  2.81958401e-47  2.72603066e-47  2.64160364e-47
  2.56518427e-47  2.49582935e-47  2.43271841e-47  2.37515080e-47
  2.32250032e-47  2.27423984e-47  2.22989274e-47  2.18905080e-47
  2.15135289e-47  2.11648133e-47  2.08416359e-47  2.05414042e-47
  2.02620290e-47  2.00015

[ 1.00000000e+00  1.00000000e+00 -9.95682444e-60 -5.97409467e-59
  3.98272978e-58 -1.59309191e-57  8.92131470e-57  6.26021397e-54
  9.86239894e-41  2.26946009e-44  7.27856072e-45  3.93483917e-45
  2.48317636e-45  1.69734768e-45  1.22089920e-45  9.10315160e-46
  6.97503810e-46  5.46286100e-46  4.35787643e-46  3.53210978e-46
  2.90334048e-46  2.41689104e-46  2.03529835e-46  1.73227610e-46
  1.48901511e-46  1.29180271e-46  1.13048826e-46  9.97461513e-47
  8.86934725e-47  7.94465761e-47  7.16604013e-47  6.50647109e-47
  5.94458749e-47  5.46339235e-47  5.04924808e-47  4.69111537e-47
  4.38002442e-47  4.10863754e-47  3.87094195e-47  3.66192578e-47
  3.47746041e-47  3.31407283e-47  3.16885329e-47  3.03935692e-47
  2.92352191e-47  2.81958401e-47  2.72603066e-47  2.64160364e-47
  2.56518427e-47  2.49582935e-47  2.43271841e-47  2.37515080e-47
  2.32250032e-47  2.27423984e-47  2.22989274e-47  2.18905080e-47
  2.15135289e-47  2.11648133e-47  2.08416359e-47  2.05414042e-47
  2.02620290e-47  2.00015

In [14]:
print(Networks_2G[0].freq)

[   1.            2.00909091    3.01818182    4.02727273    5.03636364
    6.04545455    7.05454545    8.06363636    9.07272727   10.08181818
   11.09090909   12.1          13.10909091   14.11818182   15.12727273
   16.13636364   17.14545455   18.15454545   19.16363636   20.17272727
   21.18181818   22.19090909   23.2          24.20909091   25.21818182
   26.22727273   27.23636364   28.24545455   29.25454545   30.26363636
   31.27272727   32.28181818   33.29090909   34.3          35.30909091
   36.31818182   37.32727273   38.33636364   39.34545455   40.35454545
   41.36363636   42.37272727   43.38181818   44.39090909   45.4
   46.40909091   47.41818182   48.42727273   49.43636364   50.44545455
   51.45454545   52.46363636   53.47272727   54.48181818   55.49090909
   56.5          57.50909091   58.51818182   59.52727273   60.53636364
   61.54545455   62.55454545   63.56363636   64.57272727   65.58181818
   66.59090909   67.6          68.60909091   69.61818182   70.62727273
   71.6363636